# CHAPTER 4 조인 · 병합 · 변형

## 목표
- 여러 테이블을 키로 연결(merge)해 사실테이블(fact)을 만든다.
- pivot_table로 보고서형 데이터(wide)를 만든다.


In [ ]:
# ============================================
# [공통] 라이브러리 / 경로 / 출력 옵션 세팅
# 이 셀은 모든 챕터 노트북에서 동일하게 사용합니다.
# ============================================

# 1) 수치 계산(NumPy) / 표 데이터(Pandas) 불러오기
import numpy as np
import pandas as pd

# 2) 파일 경로를 운영체제와 무관하게 다루기 위한 Path
from pathlib import Path

# 3) 현재 노트북이 실행되는 폴더를 기준(BASE)으로 데이터 폴더(DATA) 지정
BASE = Path(".").resolve()          # 현재 작업 폴더(절대경로)
DATA = BASE / "data"                # data 폴더 경로

# 4) Pandas 출력 옵션(교육용): 너무 길게 출력되지 않도록 적당히 제한
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 140)

# 5) 확인 출력
print("BASE:", BASE)
print("DATA exists:", DATA.exists())


## 4.1 데이터 로드

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 조인 실습용 테이블 4개를 로드합니다.

customers = pd.read_csv(DATA / "customers.csv", parse_dates=["signup_date"])
orders    = pd.read_csv(DATA / "orders.csv", parse_dates=["order_date"])
items     = pd.read_csv(DATA / "order_items.csv")
products  = pd.read_csv(DATA / "products.csv")

print("customers:", customers.shape)
print("orders   :", orders.shape)
print("items    :", items.shape)
print("products :", products.shape)

## 4.2 고객 + 주문 merge

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 주문 테이블에 고객의 region/age를 붙입니다(LEFT JOIN).

orders_c = orders.merge(
    customers[["customer_id","region","age"]],
    on="customer_id",
    how="left"
)

display(orders_c.head())

## 4.3 주문상세 + 상품 + 주문 merge → fact 테이블

In [ ]:
# [이 셀은 무엇을 하는가?]
# - order_items에 products를 붙인 뒤
# - orders_c를 붙여 최종 fact 테이블을 만들고 amount(매출)를 계산합니다.

items_p = items.merge(products, on="product_id", how="left")
fact = items_p.merge(orders_c, on="order_id", how="left")

fact["amount"] = fact["qty"] * fact["unit_price"]         # 파생지표: 매출
display(fact.head())
print("fact.shape:", fact.shape)

## 4.4 pivot_table: 일자×카테고리 매출표

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 보고서 형태(날짜 행, 카테고리 열)의 매출표를 만듭니다.

daily_cat = fact.pivot_table(
    index="order_date",
    columns="category",
    values="amount",
    aggfunc="sum",
    fill_value=0
).sort_index()

display(daily_cat.head())

out_path = DATA / "daily_category_sales.csv"
daily_cat.to_csv(out_path)
print("saved:", out_path)